# 1. What interesting information user can find in the dataset?

###### Input data summary

 A. 109 whiskies are described by 5 characteristics: color (14 options),nose (12 options),body (8 options), palate (15 options), finish (19 options).

 B. for each whisky there is distillery assigned. Distillery might be identified by geographical coordinates (latitude/longitude) and region/district (which indicates macrogeographic location, such as proximity to the sea, altiude etc.)

 C. for each whisky there is score indicating quality (varies from 55% to 90%): score is based on one of references invoked in "Lapointe, F.-J. & P. Legendre. 1994. A classification of pure malt Scotch whiskies." study, which is starting point for this study.

###### Identified potential data applications


 A. relation between whisky quality score and different characteristics: check if whiskies scoring higher are more likely to have certain values of color,nose, body, palate and finish ?

 B. relation between distillery location/district/region and whisky's characteristics: is there any ? (for example, if distilleries of lower altitude tend to produce whisky of certain colour ?)

 C. similarity of whiskies (being analazyed in further part if this study): considering characteristics for each whisky, how "similar" they are to each other ?

# 2. Core code: determine whiskies similarity

ASSUMPTION: since questions in the task are related to taste preferences, distillery details (location etc) are omitted.

###### Define function for whisky distance, including only one characteristic

In [52]:
import pandas as pd
from math import *

#EUCLIDEAN DISTANCE: calculates distance of vectors
def euclidean_distance(x,y):
    return sqrt(sum(pow(a-b,2) for a,b in zip(x,y)))

#DISTANCE OF ONE WHISKY TO THE ALL OTHER WHISKIES, INCLUDING ONLY ONE CHARACTERISTIC
def char_distance(charFilename,whiskyID):
    df = pd.read_csv(charFilename,delimiter = " ", header = None, skiprows=1)# read data for specified characteristics
    x = df.iloc[whiskyID]
    y = df.iloc[whiskyID+1]
    distance = []
    for idx,row in df.iterrows():
        distance.append(euclidean_distance(x,row))
    df = pd.DataFrame(distance)
    return df
print(char_distance('BODY_PSutkowski.TXT',0)) # EXAMPLE: test color similarity of whisky indexed as 1 to all whiskies

            0
0    0.000000
1    1.732051
2    2.000000
3    1.732051
4    1.000000
..        ...
104  0.000000
105  1.000000
106  1.732051
107  1.732051
108  1.732051

[109 rows x 1 columns]


###### Define function that will calculate similarity of one whisky to another, including all characteristic.

ASSUMPTION: all characteristic are considered to have even weight

In [56]:
def whiskies_distance(charList,whiskyID):
    i=0
    for characteristic in charList:
        if i==0:
            df = pd.DataFrame(char_distance(characteristic,whiskyID))# read/process first characteristic
            i = 1
        else:
            df = df.join(char_distance(characteristic,whiskyID),rsuffix=charList[i]) #read/process further characteristics
            i=i+1
    df.columns=charList # rename columns to match characteristics
    df['distance'] = df.mean(axis=1)
    df['name'] = pd.read_csv("NAMES_PSutkowski.txt",header=None) # read whiskies names from custom file
    df['score'] = pd.read_csv("SCORES_PSutkowski.txt",header=None) # read whiskies names from custom file
    return df
charList = ['COLOR.TXT','NOSE_PSutkowski.TXT','BODY_PSutkowski.TXT','PALATE.TXT','FINISH.TXT'] 

print(whiskies_distance(charList,77).sort_values(by='distance'))
    

     COLOR.TXT  NOSE_PSutkowski.TXT  BODY_PSutkowski.TXT  PALATE.TXT  \
77    0.000000             0.000000             0.000000    0.000000   
103   0.000000             1.732051             2.236068    2.000000   
98    1.414214             1.732051             1.732051    1.000000   
88    1.414214             1.000000             2.236068    1.732051   
87    0.000000             2.236068             1.732051    2.236068   
..         ...                  ...                  ...         ...   
1     1.414214             2.449490             2.000000    2.828427   
108   1.414214             2.236068             2.449490    2.828427   
83    1.414214             2.449490             2.236068    2.645751   
40    1.414214             2.449490             2.449490    2.449490   
26    1.414214             2.828427             2.645751    2.236068   

     FINISH.TXT  distance        name  score  
77     0.000000  0.000000   Lagavulin     89  
103    1.414214  1.476466   Teaninich    

# 3. Data visualization and further processing